In [0]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul 21 21:17:48 2019

@author: Junfeng Hu
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse
import matplotlib.pyplot as plt
#from resnet import ResNet18
import torch.nn.functional as F
CUDA_VISIBLE_DEVICES=(2)
# 定义是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 参数设置,使得我们能够手动输入命令行参数
if 0:
    parser = argparse.ArgumentParser(description='PyTorch MNIST Training')
    parser.add_argument('--outf', default='./model/', help='folder to output images and model checkpoints') #输出结果保存路径
    parser.add_argument('--net', default='./model/Resnet18.pth', help="path to net (to continue training)")  #恢复训练时的模型路径
    args = parser.parse_args()

# 超参数设置
EPOCH = 1   #遍历数据集次数
pre_epoch = 0  # 定义已经遍历数据集的次数
BATCH_SIZE = 32      #批处理尺寸(batch_size)
LR = 0.001        #学习率

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
         # 如果输入和输出的通道不一致，或其步长不为 1，需要将二者转成一致
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)# 输出 + 输入
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        #ResNet 包含多个 layer, 每个 layer 又包含多个 residual block (上面实现的类)
        #因此, 用 ResidualBlock 实现 Residual 部分，用 _make_layer 函数实现 layer
        #四个 layer， 对应 2， 3， 4， 5 层， 每层有两个 residual block
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        # 最后的全连接，分类时使用
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        #构建 layer, 每一个 layer 由多个 residual block 组成
        #在 ResNet 中，每一个 layer 中只有两个 residual block
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        # 最开始的处理
        out = self.conv1(x)
        # 四层 layer
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # 全连接 输出分类信息
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

def train():
    # 准备数据集并预处理
    transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32
        transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.49],
                                 std=[1])])

    trainset = torchvision.datasets.CIFAR10(root='CIFAR10/', train=True, download=True, transform=transform) #训练数据集
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)   #生成一个个batch进行批训练，组成batch的时候顺序打乱取

    testset = torchvision.datasets.CIFAR10(root='CIFAR10/', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=0)

    # 模型定义-ResNet
    net = ResNet18().cuda()

    # 定义损失函数和优化方式
    criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
    optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

    # 训练

    test_losses = []
    test_acces = []
    losses = []
    acces = []


    for epoch in range(pre_epoch, EPOCH):
        print('\nEpoch: %d' % (epoch + 1))
        net.train()
        sum_loss = 0.0
        correct = 0.0
        total = 0.0

        train_loss=0
        flag=0
        for epoch in range(EPOCH):
            for step, (inputs,labels) in enumerate(trainloader):
                inputs, labels = inputs.cuda(), labels.cuda()
                inputs = inputs.cuda()
                labels = labels.cuda()
                output = net(inputs)
                loss = criterion(output, labels)
                losses.append(loss.item())
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total = labels.size(0)
                _, predicted = torch.max(output.data, 1)
                correct = (predicted == labels).sum().item()
                acces.append(correct / total)

                #if step % 300 ==0:
                print('Epoch:', epoch, '|Step:', step,  
                  '|train loss:%.4f'%loss.data.item())
                acc,los = test(net, trainloader)
                print('Epoch', epoch, '|step ', step, 'loss: %.4f' %los.item(), 'test accuracy:%.4f' %acc)
                test_acces.append(acc)
                test_losses.append(los.item())
                if step == 120:
                  flag=1
                  break
            if flag==1:
              break
    print('Finished Training')

    plt.plot(losses)
    plt.plot(test_losses)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.show()

    plt.plot(acces)
    plt.plot(test_acces)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.show()
    
def test(net, testdata):
    criterion = nn.CrossEntropyLoss()
    correct, total = .0, .0
    net.eval()
    for inputs, labels in testdata:
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        los = criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    net.train()
    return float(correct) / total ,los

if __name__ == '__main__':
    net = train()


Files already downloaded and verified
Files already downloaded and verified

Epoch: 1
Epoch: 0 |Step: 0 |train loss:2.3657
Epoch 0 |step  0 loss: 2.3106 test accuracy:0.1000
Epoch: 0 |Step: 1 |train loss:2.3547
Epoch 0 |step  1 loss: 2.3042 test accuracy:0.1000
Epoch: 0 |Step: 2 |train loss:2.3630
Epoch 0 |step  2 loss: 2.3033 test accuracy:0.1000
Epoch: 0 |Step: 3 |train loss:2.3427
Epoch 0 |step  3 loss: 2.3091 test accuracy:0.1000
Epoch: 0 |Step: 4 |train loss:2.3178
Epoch 0 |step  4 loss: 2.3030 test accuracy:0.1000
Epoch: 0 |Step: 5 |train loss:2.3340
Epoch 0 |step  5 loss: 2.2861 test accuracy:0.1000
Epoch: 0 |Step: 6 |train loss:2.3313
Epoch 0 |step  6 loss: 2.3032 test accuracy:0.1000
Epoch: 0 |Step: 7 |train loss:2.2934
Epoch 0 |step  7 loss: 2.3051 test accuracy:0.1000
Epoch: 0 |Step: 8 |train loss:2.2917
